In [ ]:
"""
File: train_spot.ipynb
Authors: Jared Galloway
Date: 1/15/2020

This Jupyter Notebook will exemplify the workflow for simulating 
a dataset of spots with thier respective annotations - 
then train a neural network to parse out dots from background noise.

Finally, we will import an empirical flurescent microscopy image and
Predict on The image before computing the F1 score as a metric of accuracy.

This Workflow is proof of concepy that spots can be simulted 
easily and accuratly enough to train a convolutional nueral network
on the task of spot detection for a range of empitical scenerios.
"""

In [ ]:
import os
import sys

# External Dependencies
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
import time

# Internal Dependencies 
sys.path.insert(0,"../")
import scripts.networks as nets
import scripts.helpers as helpers
import scripts.dot_simulator as dot_simulator

In [ ]:
help(dot_simulator.single_layers_simulator)

In [ ]:
# The directory to place output files and location of test files
EXP_DIR = "../scripts/L1-D02-z_490_448_64"

In [ ]:
# Choose Some parameters
params = {"num_samples":1500,
            "width":64,
            "height":64,
            "num_dots_n": 0,
            "num_dots_p": 0.85,
            "radii_n":4,
            "radii_p":0.65,
            "spot_noise":0.2,
            "point_noise":0.2,
            "background_noise":0.15}

# Simulate!
start = time.time()
x, y = dot_simulator.single_layers_simulator(**params)
finish = time.time()
print(f"Simulations took {round(finish - start,2)} seconds")

print(f"simulated data examples has shape: {x.shape}")
print(f"simulated data set targets has shape: {y.shape}")

In [ ]:
# Let's see what (roughly) our distribution of dots, and radii will be ...
# dots = np.random.binomial(n = params["num_dots_n"], p = params["num_dots_p"], size = params["num_samples"])
dots = np.random.binomial(n = params["num_dots_n"], p = params["num_dots_p"], size = 10000)
radii = np.random.binomial(n = params["radii_n"], p = params["radii_p"], size = sum(dots))
fig, ax = plt.subplots(1,2, figsize=(14, 7))
ax[0].hist(dots)
ax[0].set_title("Exp Number of Dots across Images")
ax[1].hist(radii)
ax[1].set_title("Exp Size of radii across Spots")

In [ ]:
IMAGE_INDEX = 0
fig, ax = plt.subplots(1,2,figsize=(14, 11))
ax[0].imshow(np.squeeze(x[IMAGE_INDEX]))
ax[1].imshow(np.squeeze(y[IMAGE_INDEX]))

In [ ]:
# cut data set up into train, validation, and testing.

percent_test = 0.1
percent_validation = 0.2

test_split = int(x.shape[0] * percent_test) 
vali_split = int(x.shape[0] * percent_validation)

test_x = x[:test_split,:,:,:]
vali_x = x[test_split:vali_split,:,:,:]
train_x = x[vali_split:,:,:,:]

test_y = y[:test_split,:,:,:]
vali_y = y[test_split:vali_split,:,:,:]
train_y = y[vali_split:,:,:,:]

In [ ]:
# choose a model!
model = nets.deeper_direct_CNN(x,y)
print(model.summary())

In [ ]:
# fit the model with our simulated data
model.fit(train_x, train_y, 
        validation_data = (vali_x, vali_y),
        epochs = 2)

In [ ]:
# Load an empirical image for testing!
test_emp_image = np.load(f"{EXP_DIR}/{EXP_DIR[11:]}_image_scaled.out", allow_pickle = True)
test_emp_image = np.reshape(test_emp_image,(1,64,64,1))

In [ ]:
# Lets take a look at the empirical image
# TODO : Take a look at the entire Image.
fig, ax = plt.subplots(1, figsize=(7, 6.5))
ax.imshow(np.squeeze(test_emp_image))

In [ ]:
# Finally, lets use our model trained on simulations to predict on the real thing!
pred_emp = model.predict(test_emp_image)
pred_sim = model.predict(test_x)

In [ ]:
# Lets take a look!
fig, ax = plt.subplots(1,2, figsize=(14, 13))
ax[0].imshow(np.squeeze(test_emp_image))
ax[1].imshow(np.squeeze(pred_emp))